## Download the Sample Data

In [ ]:
# The tar file is 9.3GB and, when extracted, an additional 14GB
!mkdir sample_data
%cd sample_data
!wget -O parabricks_sample.tar.gz "https://s3.amazonaws.com/parabricks.sample/parabricks_sample.tar.gz"
!tar xvf parabricks_sample.tar.gz
!mv parabricks_sample/* .
%cd ..

In [2]:
!ls sample_data

Data  Ref


In [3]:
!ls sample_data/Data

sample_1.fq.gz	sample_2.fq.gz


In [4]:
!ls sample_data/Ref

Homo_sapiens_assembly38.dict
Homo_sapiens_assembly38.fasta
Homo_sapiens_assembly38.fasta.amb
Homo_sapiens_assembly38.fasta.ann
Homo_sapiens_assembly38.fasta.bwt
Homo_sapiens_assembly38.fasta.fai
Homo_sapiens_assembly38.fasta.pac
Homo_sapiens_assembly38.fasta.sa
Homo_sapiens_assembly38.known_indels.vcf.gz
Homo_sapiens_assembly38.known_indels.vcf.gz.tbi


## GPU Monitoring

In [9]:
!nvidia-smi

Wed Nov 29 07:06:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:07:00.0 Off |                  Off |
| 33%   34C    P8              15W / 450W |      2MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Run the command below in the terminal
### watch -n 0.5 nvidia-smi
#

## Alignment: FASTQ to BAM

In [6]:
!pbrun fq2bam -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


usage: pbrun fq2bam [<args>]
Help: pbrun fq2bam -h

Run GPU-bwa mem, co-ordinate sorting, marking duplicates, and Base Quality
Score Recalibration to convert FASTQ to BAM/CRAM.

optional arguments:
  -h, --help            show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool:

  --ref REF             Path to the reference file. (default: None)
  --in-fq [IN_FQ [IN_FQ ...]]
                        Path to the pair-ended FASTQ files followed by
                        optional read groups with quotes (Example:
                        "@RG\tID:foo\tLB:lib1\tPL:bar\tSM:sample\tPU:foo").
                        The files must be in fastq or fastq.gz format. All
                        sets of inputs should have a read group; otherwise,
                        none should have a read group, and it will be
                        automatically added by t

In [8]:
!pbrun fq2bam \
      --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
      --in-fq /workdir/sample_data/Data/sample_1.fq.gz /workdir/sample_data/Data/sample_2.fq.gz \
      --out-bam /workdir/outputdir/fq2bam_output.bam
      --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation



[Parabricks Options Mesg]: Checking argument compatibility
[Parabricks Options Mesg]: Automatically generating ID prefix
[Parabricks Options Mesg]: Read group created for /workdir/sample_data/Data/sample_1.fq.gz and
/workdir/sample_data/Data/sample_2.fq.gz
[Parabricks Options Mesg]: @RG\tID:HK3TJBCX2.1\tLB:lib1\tPL:bar\tSM:sample\tPU:HK3TJBCX2.1
[PB Info 2023-Nov-29 07:01:32] ------------------------------------------------------------------------------
[PB Info 2023-Nov-29 07:01:32] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2023-Nov-29 07:01:32] ||                              Version 4.1.1-1                             ||
[PB Info 2023-Nov-29 07:01:32] ||                       GPU-BWA mem, Sorting Phase-I                       ||
[PB Info 2023-Nov-29 07:01:32] ------------------------------------------------------------------------------
[M::bwa_idx_load_

In [10]:
!ls outputdir/

fq2bam_output.bam  fq2bam_output.bam.bai  fq2bam_output_chrs.txt


## Variant Calling

#### GATK Haplotypecaller

In [11]:
!pbrun haplotypecaller -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

usage: pbrun haplotypecaller <options>
Help: pbrun haplotypecaller -h

Run HaplotypeCaller to convert BAM/CRAM to VCF.

optional arguments:
  -h, --help            show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool:

  --ref REF             Path to the reference file. (default: None)
  --in-bam IN_BAM       Path to the input BAM/CRAM file for variant calling.
                        The argument may also be a local folder containing
                        several bams; each will be processed by 1 GPU in batch
                        mode. (default: None)
  --in-recal-file IN_RECAL_FILE
                        Path to the input BQSR report. (default: None)
  --interval-file INTERVAL_FILE
                        Path to an interval file in one of these formats:
                        Picard-style (.interval_list or .picard), GATK-style
         

- vcf

In [12]:
!pbrun haplotypecaller \
      --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
      --in-bam /workdir/outputdir/fq2bam_output.bam \
      --out-variants /workdir/outputdir/variants_gatk.vcf \
      --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

/usr/local/parabricks/binaries//bin/htvc /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 -o /workdir/outputdir/variants_gatk.vcf -nt 5
[PB Info 2023-Nov-29 07:23:17] ------------------------------------------------------------------------------
[PB Info 2023-Nov-29 07:23:17] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2023-Nov-29 07:23:17] ||                              Version 4.1.1-1                             ||
[PB Info 2023-Nov-29 07:23:17] ||                         GPU-GATK4 HaplotypeCaller                        ||
[PB Info 2023-Nov-29 07:23:17] ------------------------------------------------------------------------------
[PB Info 2023-Nov-29 07:23:34] 0 /workdir/outputdir/fq2bam_output.bam/workdir/outputdir/variants_gatk.vcf
[PB Info 2023-Nov-29 07:23:34] ProgressMeter -	Current-Locus	Elapsed-Minutes	Reg

- gvcf

In [14]:
!pbrun haplotypecaller \
      --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
      --in-bam /workdir/outputdir/fq2bam_output.bam \
      --gvcf \
      --out-variants /workdir/outputdir/variants_gatk.g.vcf \
      --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

/usr/local/parabricks/binaries//bin/htvc /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 -o /workdir/outputdir/variants_gatk.gvcf -nt 5 -g
[PB Info 2023-Nov-29 07:36:46] ------------------------------------------------------------------------------
[PB Info 2023-Nov-29 07:36:46] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2023-Nov-29 07:36:46] ||                              Version 4.1.1-1                             ||
[PB Info 2023-Nov-29 07:36:46] ||                         GPU-GATK4 HaplotypeCaller                        ||
[PB Info 2023-Nov-29 07:36:46] ------------------------------------------------------------------------------
[PB Info 2023-Nov-29 07:37:03] 0 /workdir/outputdir/fq2bam_output.bam/workdir/outputdir/variants_gatk.gvcf
[PB Info 2023-Nov-29 07:37:03] ProgressMeter -	Current-Locus	Elapsed-Minute

"The key difference between a regular VCF and a GVCF is that the GVCF has records for all sites, whether there is a variant call there or not. The goal is to have every site represented in the file in order to do joint analysis of a cohort in subsequent steps." (https://gatk.broadinstitute.org/hc/en-us/articles/360035531812-GVCF-Genomic-Variant-Call-Format)

In [15]:
!ls -al -h outputdir/

total 6.9G
drwxr-xr-x. 2 root root  225 Nov 29 07:45 .
drwxr-xr-x. 5 root root  216 Nov 29 07:46 ..
-rw-r--r--. 1 root root 4.5G Nov 29 07:05 fq2bam_output.bam
-rw-r--r--. 1 root root 6.6M Nov 29 07:05 fq2bam_output.bam.bai
-rw-r--r--. 1 root root  86K Nov 29 07:05 fq2bam_output_chrs.txt
-rw-r--r--. 1 root root 1.6G Nov 29 07:45 variants_gatk.gvcf
-rw-r--r--. 1 root root 389K Nov 29 07:45 variants_gatk.gvcf.idx
-rw-r--r--. 1 root root  23M Nov 29 07:27 variants_gatk.vcf


#### DeepVariant

In [16]:
!pbrun deepvariant -h

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

usage: pbrun deepvariant <options>
Help: pbrun deepvariant -h

Run DeepVariant to convert BAM/CRAM to VCF.

optional arguments:
  -h, --help            show this help message and exit

Input Output file options:
  Options for Input and Output files for this tool:

  --ref REF             Path to the reference file. (default: None)
  --in-bam IN_BAM       Path to the input BAM/CRAM file for variant calling.
                        (default: None)
  --interval-file INTERVAL_FILE
                        Path to a BED file (.bed) for selective access. This
                        option can be used multiple times. (default: None)
  --out-variants OUT_VARIANTS
                        Path of the vcf/g.vcf/g.vcf.gz file after variant
                        calling. (default: None)
  --pb-model-file PB_MODEL_FILE
                        Path to a non-default parabricks model file for
                        de

- vcf

In [18]:
!pbrun deepvariant \
    --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-bam /workdir/outputdir/fq2bam_output.bam \
    --out-variants /workdir/outputdir/variants_dv.vcf \
    --num-streams-per-gpu 2 \
    --run-partition \
    --gpu-num-per-partition 1 \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.2 / 12.0
Using model for CUDA Capability Major/Minor version number:    89
CUDA_VISIBLE_DEVICES=0 /usr/local/parabricks/binaries//bin/deepvariant /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 2 -o 0.vcf  -L "chr1" -L "chr2" -L "chr3" -L "chr4" -L "chr5" -L "chr6" -L "chr7" -L "chrX" -L "chr8" -L "chr9" -L "chr11" -L "chr10" -L "chr12" -L "chr13" -L "chr14" -L "chr15" -L "chr16" -L "chr17" -L "chr18" -L "chr20" -L "chr19" -L "chrY" -L "chr22" -L "chr21" -L "chrM" -n 6 --model /usr/local/parabricks/binaries//model/80+/shortread/deepvariant.eng --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_bas

DeepVariant from Parabricks has the ability to use multiple streams on a GPU. The number of streams that can be used depends on the available resources. The default number of streams is set to two but can be increased up to a maximum of six to get better performance. This is something that has to be experimented with, before getting the optimal number on your system. (https://docs.nvidia.com/clara/parabricks/4.1.0/bestperformance.html#best-performance-for-deepvariant)

In [21]:
!pbrun deepvariant \
    --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-bam /workdir/outputdir/fq2bam_output.bam \
    --out-variants /workdir/outputdir/variants_dv.vcf \
    --num-streams-per-gpu 4 \
    --run-partition \
    --gpu-num-per-partition 1 \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.2 / 12.0
Using model for CUDA Capability Major/Minor version number:    89
CUDA_VISIBLE_DEVICES=0 /usr/local/parabricks/binaries//bin/deepvariant /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 4 -o 0.vcf  -L "chr1" -L "chr2" -L "chr3" -L "chr4" -L "chr5" -L "chr6" -L "chr7" -L "chrX" -L "chr8" -L "chr9" -L "chr11" -L "chr10" -L "chr12" -L "chr13" -L "chr14" -L "chr15" -L "chr16" -L "chr17" -L "chr18" -L "chr20" -L "chr19" -L "chrY" -L "chr22" -L "chr21" -L "chrM" -n 6 --model /usr/local/parabricks/binaries//model/80+/shortread/deepvariant.eng --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_bas

- gvcf

Using the --run-partition, --proposed-variants, and --gvcf options at the same time will lead to a substantial slowdown.

In [24]:
!pbrun deepvariant \
    --ref /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-bam /workdir/outputdir/fq2bam_output.bam \
    --out-variants /workdir/outputdir/variants_dv.g.vcf \
    --num-streams-per-gpu 4 \
    --gvcf \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

Detected 1 CUDA Capable device(s), considering 1 device(s)
  CUDA Driver Version / Runtime Version          12.2 / 12.0
Using model for CUDA Capability Major/Minor version number:    89
/usr/local/parabricks/binaries//bin/deepvariant /workdir/sample_data/Ref/Homo_sapiens_assembly38.fasta /workdir/outputdir/fq2bam_output.bam 1 4 -o /workdir/outputdir/variants_dv.g.vcf -n 6 --model /usr/local/parabricks/binaries//model/80+/shortread/deepvariant.eng -g --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_base_quality 10 --alt_aligned_pileup none --variant_caller VERY_SENSITIVE_CALLER --dbg_min_base_quality 15 --ws_min_windows_distance 80 --aux_fields_to_keep HP --p_error 0.001 --max_ins_size 10
[PB Info 2023-Nov-30 03:38:13] -------------------------------